# Visuallize KG recheck

## Load generated summary KG

In [1]:
from pathlib import Path
import json
import pandas as pd

main_path = str((Path().absolute()))
models = [
    'bart-large', 
    'bart-large-cnn'
]
experiments = [
    ['abstract-kg', 'abstract-kg'],
    ['summary-kg', 'abstract-kg'],
    ['summary-kg', 'summary-kg'],
]
best_score = ['output_bestRouge1', 'output_bestRougeAvg']
exclude = [
    'bart-large-cnn_abstract-kg_abstract-kg',
    'bart-large-cnn_summary-kg_summary-kg',
]
def load_re(filename):
    filepath = f"_scire_models/csv/{filename}_re.json"
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    return [json.loads(json_str) for json_str in json_list]

def load_pre(filename):
    filepath = f"_prepared_data/csv/{filename}.jsonl"
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    return [json.loads(json_str) for json_str in json_list]


In [71]:
result_all = []
for model in models:
    for exp in experiments:
        for best in best_score:
            if f"{model}_{exp[0]}_{exp[1]}" in exclude: 
                filename=f"[{model}]_[{exp[0]}]_[{exp[1]}]"
            else:
                filename=f"[{model}]_[{exp[0]}]_[{exp[1]}]_{best[11:]}"
                
            try:
                result_list = load_re(filename)
                print(len(result_all), filename, len(result_list))
                ner = re = 0
                for result in result_list:
                    if len(result['predicted_ner'])==0: ner+=1
                    if len(result['predicted_re'])==0: re+=1
                print(f"\tBlank NER {ner}/{len(result_list)}")
                print(f"\tBlank RE {re}/{len(result_list)}")
                result_all.append(result_list)
            except:
                print(filename, "CANNOT IMPORT")
    #         break
    #     break
    # break
            

0 [bart-large]_[abstract-kg]_[abstract-kg]_Rouge1 1060
	Blank NER 0/1060
	Blank RE 0/1060
1 [bart-large]_[abstract-kg]_[abstract-kg]_RougeAvg 1060
	Blank NER 0/1060
	Blank RE 0/1060
2 [bart-large]_[summary-kg]_[abstract-kg]_Rouge1 1060
	Blank NER 0/1060
	Blank RE 0/1060
3 [bart-large]_[summary-kg]_[abstract-kg]_RougeAvg 1060
	Blank NER 0/1060
	Blank RE 0/1060
4 [bart-large]_[summary-kg]_[summary-kg]_Rouge1 3604
	Blank NER 0/3604
	Blank RE 3599/3604
5 [bart-large]_[summary-kg]_[summary-kg]_RougeAvg 3604
	Blank NER 0/3604
	Blank RE 3600/3604
6 [bart-large-cnn]_[abstract-kg]_[abstract-kg] 1060
	Blank NER 0/1060
	Blank RE 0/1060
7 [bart-large-cnn]_[abstract-kg]_[abstract-kg] 1060
	Blank NER 0/1060
	Blank RE 0/1060
8 [bart-large-cnn]_[summary-kg]_[abstract-kg]_Rouge1 1060
	Blank NER 0/1060
	Blank RE 1057/1060
9 [bart-large-cnn]_[summary-kg]_[abstract-kg]_RougeAvg 1060
	Blank NER 0/1060
	Blank RE 1057/1060
10 [bart-large-cnn]_[summary-kg]_[summary-kg] 3604
	Blank NER 0/3604
	Blank RE 3600/36

## Load reference summary KG

In [73]:
def load_refKG():
    filepath = "_scire_models/MuP/summary/val_re.json"
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    return [json.loads(json_str) for json_str in json_list]

def evaluate(gen):
    drop_col = ["ner", "relations", "predicted_re"]
    ref = pd.DataFrame(load_refKG()).drop(drop_col, axis=1).rename(columns={"predicted_ner": "ref_ner", "sentences": "ref_sentences"})
    gen = gen.drop(drop_col, axis=1).rename(columns={"predicted_ner": "gen_ner", "sentences": "gen_sentences"})
    # return pd.concat([gen, ref.drop("doc_key", axis=1)], axis=1)
    return pd.merge(gen, ref, on='doc_key', how='outer')

In [133]:
df_kg = evaluate(pd.DataFrame(result_all[4]))
df_kg.iloc[:20]

,doc_key,gen_sentences,gen_ner,ref_sentences,ref_ner
0,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, studies, the, problem, of, priv...","[[[6, 10, Task], [12, 15, OtherScientificTerm]...","[[This, paper, studies, FL, under, local, diff...","[[[3, 3, Task], [5, 8, OtherScientificTerm]], ..."
1,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, studies, the, problem, of, priv...","[[[6, 10, Task], [12, 15, OtherScientificTerm]...","[[This, paper, studies, a, low, communication,...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate..."
2,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, studies, the, problem, of, priv...","[[[6, 10, Task], [12, 15, OtherScientificTerm]...","[[The, paper, proposed, a, differentially, pri...","[[[4, 7, Method], [9, 10, Method]], [[17, 18, ..."
3,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, proposes, a, low, communication...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate...","[[This, paper, studies, FL, under, local, diff...","[[[3, 3, Task], [5, 8, OtherScientificTerm]], ..."
4,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, proposes, a, low, communication...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate...","[[This, paper, studies, a, low, communication,...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate..."
5,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, proposes, a, low, communication...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate...","[[The, paper, proposed, a, differentially, pri...","[[[4, 7, Method], [9, 10, Method]], [[17, 18, ..."
6,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, proposes, a, differentially, pr...","[[[4, 7, Method], [9, 10, Method]], [[19, 22, ...","[[This, paper, studies, FL, under, local, diff...","[[[3, 3, Task], [5, 8, OtherScientificTerm]], ..."
7,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, proposes, a, differentially, pr...","[[[4, 7, Method], [9, 10, Method]], [[19, 22, ...","[[This, paper, studies, a, low, communication,...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate..."
8,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,"[[This, paper, proposes, a, differentially, pr...","[[[4, 7, Method], [9, 10, Method]], [[19, 22, ...","[[The, paper, proposed, a, differentially, pri...","[[[4, 7, Method], [9, 10, Method]], [[17, 18, ..."
9,SP:632666b52c7c551d67fbbe70c06ed589c3a5e187,"[[This, paper, studies, the, neural, machine, ...","[[[4, 7, Task], [12, 17, Method]], [[23, 24, M...","[[This, submission, works, on, the, neural, ma...","[[[5, 8, Task]], [[18, 19, OtherScientificTerm..."


In [134]:
def ext_ent(data, sent_col, ner_col):
    all_sentences = [j for i in data[sent_col] for j in i]
    flatten_ner = [j for i in data[ner_col] for j in i]
    return [" ".join(all_sentences[ner[0]:ner[1]+1]) for ner in flatten_ner]

def cal_RP(gen, ref):
    c = 0    
    for ele in gen: 
        if ele in ref: c+=1
    recall = c/len(ref)*100
    precision = c/len(gen)*100
    return recall, precision
        
def eval_ent(data):
    gen_ner = list(set(ext_ent(data, 'gen_sentences', 'gen_ner')))
    ref_ner = list(set(ext_ent(data, 'ref_sentences', 'ref_ner')))   
    return cal_RP(gen_ner, ref_ner)
    # print(f"Recall: {recall:.2f} %")
    # print(f"Precision: {precision:.2f} %")

r_list = []
p_list = []
for i in range(14):
    recall, precision = eval_ent(df_kg.iloc[i])
    r_list.append(recall)
    p_list.append(precision)

In [135]:
print(r_list[:10])
print(p_list[:10])

[81.81818181818183, 12.5, 7.142857142857142, 18.181818181818183, 87.5, 21.428571428571427, 9.090909090909092, 18.75, 64.28571428571429, 68.42105263157895]
[90.0, 20.0, 10.0, 14.285714285714285, 100.0, 21.428571428571427, 11.11111111111111, 33.33333333333333, 100.0, 92.85714285714286]


In [136]:
print(sum(r_list)/len(r_list))
print(sum(p_list)/len(p_list))

33.62851903735363
47.97052154195011


In [29]:
gen_sentences = (df_kg['gen_sentences'][0])

[['This', 'paper', 'studies', 'the', 'problem', 'of', 'local', 'differential', 'privacy', 'constraints', 'in', 'FL', '.'], ['The', 'authors', 'propose', 'a', 'gradient', 'subsampling', 'strategy', 'to', 'reduce', 'the', 'communication', 'costs', 'and', 'improve', 'privacy', '-', 'preserving', 'FL', 'algorithms', '.'], ['The', 'main', 'contribution', 'of', 'the', 'paper', 'is', 'to', 'propose', 'a', 'new', 'gradient', '-', 'based', 'learning', 'algorithm', 'sqSGD', '.'], ['The', 'key', 'idea', 'is', 'to', 'apply', 'randomized', 'rotation', 'to', 'the', 'quantization', 'error', 'of', 'the', 'base', 'algorithm', 'to', 'improve', 'the', 'communication', 'efficiency', 'and', 'highdimensional', 'compatibility', '.']]


In [27]:
df_kg['gen_ner'][0]

[[[6, 9, 'Task'], [11, 11, 'Task']],
 [[17, 19, 'Method'], [23, 24, 'Metric'], [27, 31, 'Method']],
 [[44, 49, 'Method']],
 [[57, 58, 'OtherScientificTerm'],
  [61, 62, 'OtherScientificTerm'],
  [65, 66, 'Generic'],
  [70, 71, 'Metric'],
  [73, 74, 'Metric']]]

In [8]:
df = pd.DataFrame(result_list)
print(df['sentences'][4])

[['This', 'paper', 'proposes', 'a', 'new', 'method', 'to', 'incorporate', 'prior', 'knowledge', 'into', 'Transformer', '-', 'based', 'sentence', 'encoders', 'for', 'neural', 'machine', 'translation', '(', 'NMT', ')', '.'], ['The', 'prior', 'knowledge', 'matrix', 'is', 'constructed', 'from', 'the', 'query', 'matrix', 'of', 'the', 'NMT', 'component', ',', 'and', 'this', 'matrix', 'is', 'used', 'to', 'encode', 'the', 'prior', 'knowledge', 'of', 'the', 'Transformer', 'encoder', '.'], ['A', 'gating', 'mechanism', 'is', 'also', 'introduced', 'to', 'control', 'the', 'quality', 'of', 'Transformer', 'decoder', 'output', '.'], ['Experiments', 'show', 'that', 'the', 'proposed', 'procedure', 'is', 'able', 'to', 'learn', 'a', 'prior', 'knowledge', 'representation', 'matrix', 'that', 'can', 'be', 'used', 'to', 'improve', 'the', 'performance', 'of', 'a', 'Transformer', 'self', '-', 'attention', 'mechanism', '.']]


In [43]:
from sklearn.metrics import confusion_matrix
import pandas as pd

# true labels
y_true = ["PERSON", "ORG", "LOCATION", "O", "PERSON", "LOCATION"]

# predicted labels
y_pred = ["PERSON", "ORG", "LOCATION", "PERSON", "O", "LOCATION"]

# create confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=["PERSON", "ORG", "LOCATION", "O"])

# convert confusion matrix to dataframe for better visualization
df_cm = pd.DataFrame(cm, index=["PERSON", "ORG", "LOCATION", "O"], columns=["PERSON", "ORG", "LOCATION", "O"])

# print confusion matrix
print(df_cm)

          PERSON  ORG  LOCATION  O
PERSON         1    0         0  1
ORG            0    1         0  0
LOCATION       0    0         2  0
O              1    0         0  0


In [47]:
# import matplotlib.pyplot as plt
# from sklearn.datasets import make_classification
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# X, y = make_classification(random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# clf = SVC(random_state=0)
# clf.fit(X_train, y_train)
# SVC(random_state=0)
# predictions = clf.predict(X_test)
cm = confusion_matrix(y_true, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

plt.show()

/home/nopphawann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:276: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all([l not in y_true for l in labels]):


ValueError: At least one label specified must be in y_true

In [48]:
predictions

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0])

In [49]:
y_test

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0])